In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install joblib=1.2.0

ERROR: Invalid requirement: 'joblib=1.2.0'
Hint: = is not a valid operator. Did you mean == ?


In [3]:

# set project folder directory
DATA_DIR = '/content/drive/MyDrive/W281 Project/data'

In [4]:
# load some lib
import os
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
import joblib

# import some modeling libraries
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16 # Load in based VGG16 model
# from keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA


# read in metadata
df_meta = pd.read_csv(f"{DATA_DIR}/raw/Chest_xray_Corona_Metadata.csv", index_col = 0)
df_meta

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...
5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [5]:
# read in summary of data set
df_data_summary = pd.read_csv(f"{DATA_DIR}/raw/Chest_xray_Corona_dataset_Summary.csv", index_col = 0)
df_data_summary['Label_2_Virus_category'] = np.where((df_data_summary['Label'] == 'Normal'), 
                                              'Normal',
                                              np.where(
                                                  (df_data_summary['Label_2_Virus_category'] != df_data_summary['Label_2_Virus_category']) & 
                                                  ~(df_data_summary['Label_1_Virus_category'] != df_data_summary['Label_1_Virus_category']),
                                                  df_data_summary['Label_1_Virus_category'],
                                                  df_data_summary['Label_2_Virus_category']
                                              ))

df_data_summary['Label_1_Virus_category'] = np.where((df_data_summary['Label'] == 'Normal'), 
                                              'Normal', 
                                              df_data_summary['Label_1_Virus_category']
                                              )

df_data_summary

,Label,Label_1_Virus_category,Label_2_Virus_category,Image_Count
0,Normal,Normal,Normal,1576
1,Pnemonia,Stress-Smoking,ARDS,2
2,Pnemonia,Virus,Virus,1493
3,Pnemonia,Virus,COVID-19,58
4,Pnemonia,Virus,SARS,4
5,Pnemonia,bacteria,bacteria,2772
6,Pnemonia,bacteria,Streptococcus,5


In [6]:
# creating data label map
df_data_summary = df_data_summary[df_data_summary['Label_1_Virus_category'] != 'Stress-Smoking']
label_map = df_data_summary[['Label_1_Virus_category']].drop_duplicates().reset_index(drop=True).to_dict()['Label_1_Virus_category']
label_map = { val.lower(): key for key, val in label_map.items()}
label_map

{'normal': 0, 'virus': 1, 'bacteria': 2}

# Image Visualization
Read in some image to show some visualization

In [ ]:
# load image data
def grab_one_and_plot(df, col_category, data_set = 'train'):
  """
    For data visualization, grab each image from the specified column directory
    Show the image
  """
  # copy the meta; avoid mutation
  df_copy = df.copy()


  # instantitae canvas for plotting
  plt.figure(figsize=(15,5))

  for i, cat in enumerate(df_copy[col_category].unique()):

      print(cat)

      # locate the image and build path to the image
      img_meta = df_copy[df_copy[col_category] == cat].iloc[153]
      train_test = data_set
      path = f"{DATA_DIR}/raw/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/{train_test}/{img_meta['X_ray_image_name']}"


      # grab the image path and plot
      img = Image.open(path)
      plt.subplot(1, len(df_copy[col_category].unique()), i+1)
      plt.imshow(img, cmap='gray')
      plt.axis('off')
      plt.title(f"{train_test}: {cat} Chest X-ray")
      
      
  plt.show()

# grab_one_and_plot(df_meta, "Label")


## Image Resizing

Interesting, the X-ray images are not of even size. Here we want to resize each image to (224 x 224) because images of even dimension are easier to model.



In [ ]:

# def resize_image(image):
#     """
#         Resize Image to fit VGG16 Imagenet input dimension
#         dim = (224 x 224 x 3)
#     """
#     width, height = image.size
#     if width > height:
#         left = (width - height) / 2
#         right = width - left
#         top = 0
#         bottom = height
#     else:
#         top = (height - width) / 2
#         bottom = height - top
#         left = 0
#         right = width
#     image = image.crop((left, top, right, bottom))
#     image = image.resize([224, 224], Image.LANCZOS)
#     return image


# def main(path_in, path_out):
#     """
#       Function to initiate the resizing process
#     """
#     if not os.path.exists(path_out):
#         os.makedirs(path_out)

#     print('Start resizing images.')
#     image_files = os.listdir(path_in)
#     num_images = len(image_files)
#     for i, image_file in tqdm(enumerate(image_files)):
#         with open(os.path.join(path_in, image_file), 'r+b') as f:
#             with Image.open(f) as image:
#                 image = resize_image(image)
#                 image.save(os.path.join(path_out, image_file), image.format)


# path_in = f'{DATA_DIR}/raw/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'
# path_out = f'{DATA_DIR}/raw/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test_resized'
# main(path_in, path_out)

In [ ]:
# grab_one_and_plot(df_meta, "Label", "train_resized")

# Simple Model

1. pre-process the image to grab the label per image
2. do a simple model with multiple labels

In [7]:
df_meta = df_meta[df_meta['Label_1_Virus_category'] != 'Stress-Smoking']

In [8]:
df_meta['Label_1_Virus_category'].unique()

array([nan, 'Virus', 'bacteria'], dtype=object)

In [9]:
# fill some label nan
# certain bacteria/virus does not have a distinct label, for these we will be using the regular virus/barteria term
# in case of normal, we will label it as nomal
df_meta['Label_2_Virus_category'] = np.where((df_meta['Label'] == 'Normal'), 
                                              'Normal',
                                              np.where(
                                                  (df_meta['Label_2_Virus_category'] != df_meta['Label_2_Virus_category']) & 
                                                  ~(df_meta['Label_1_Virus_category'] != df_meta['Label_1_Virus_category']),
                                                  df_meta['Label_1_Virus_category'],
                                                  df_meta['Label_2_Virus_category']
                                              ))


df_meta['Label_1_Virus_category'] = np.where((df_meta['Label'] == 'Normal'), 
                                              'Normal',
                                              df_meta['Label_1_Virus_category'])

# select and rename meta dataset columns
df_meta = df_meta[['Dataset_type', 'X_ray_image_name', 'Label_1_Virus_category']]
df_meta['Label_1_Virus_category'] = df_meta['Label_1_Virus_category'].str.lower()
df_meta['label'] = df_meta['Label_1_Virus_category'].map(label_map)


# get a list of unique labels to train from
label_list = sorted(list(df_meta['label'].unique()))
print(label_list)

# get train data
df_train = df_meta[df_meta['Dataset_type'] == 'TRAIN']
df_train

[0, 1, 2]


<ipython-input-9-080ed7f18449>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta['Label_2_Virus_category'] = np.where((df_meta['Label'] == 'Normal'),
<ipython-input-9-080ed7f18449>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta['Label_1_Virus_category'] = np.where((df_meta['Label'] == 'Normal'),


,Dataset_type,X_ray_image_name,Label_1_Virus_category,label
0,TRAIN,IM-0128-0001.jpeg,normal,0
1,TRAIN,IM-0127-0001.jpeg,normal,0
2,TRAIN,IM-0125-0001.jpeg,normal,0
3,TRAIN,IM-0122-0001.jpeg,normal,0
4,TRAIN,IM-0119-0001.jpeg,normal,0
...,...,...,...,...
5304,TRAIN,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,virus,1
5305,TRAIN,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,virus,1
5306,TRAIN,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,virus,1
5307,TRAIN,1-s2.0-S0140673620303706-fx1_lrg.jpg,virus,1


In [10]:
# get test data
df_test = df_meta[df_meta['Dataset_type'] == 'TEST']
df_test

,Dataset_type,X_ray_image_name,Label_1_Virus_category,label
5309,TEST,IM-0021-0001.jpeg,normal,0
5310,TEST,IM-0019-0001.jpeg,normal,0
5311,TEST,IM-0017-0001.jpeg,normal,0
5312,TEST,IM-0016-0001.jpeg,normal,0
5313,TEST,IM-0015-0001.jpeg,normal,0
...,...,...,...,...
5928,TEST,person1637_virus_2834.jpeg,virus,1
5929,TEST,person1635_virus_2831.jpeg,virus,1
5930,TEST,person1634_virus_2830.jpeg,virus,1
5931,TEST,person1633_virus_2829.jpeg,virus,1


In [11]:
df_train['IMG'] = df_train['X_ray_image_name'].str.split(".").str[0]
df_test['IMG'] = df_test['X_ray_image_name'].str.split(".").str[0]

<ipython-input-11-e47e5c6574bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['IMG'] = df_train['X_ray_image_name'].str.split(".").str[0]
<ipython-input-11-e47e5c6574bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['IMG'] = df_test['X_ray_image_name'].str.split(".").str[0]


In [ ]:
df_train

,Dataset_type,X_ray_image_name,Label_1_Virus_category,label,IMG
0,TRAIN,IM-0128-0001.jpeg,normal,0,IM-0128-0001
1,TRAIN,IM-0127-0001.jpeg,normal,0,IM-0127-0001
2,TRAIN,IM-0125-0001.jpeg,normal,0,IM-0125-0001
3,TRAIN,IM-0122-0001.jpeg,normal,0,IM-0122-0001
4,TRAIN,IM-0119-0001.jpeg,normal,0,IM-0119-0001
...,...,...,...,...,...
5304,TRAIN,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,virus,1,1-s2
5305,TRAIN,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,virus,1,1-s2
5306,TRAIN,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,virus,1,1-s2
5307,TRAIN,1-s2.0-S0140673620303706-fx1_lrg.jpg,virus,1,1-s2


In [12]:
import pickle
hsv_features_train = pickle.load(open(f"{DATA_DIR}/processed/hsv_hist_features_train.pkl", "rb"))
hsv_features_test = pickle.load(open(f"{DATA_DIR}/processed/hsv_hist_features_test.pkl", "rb"))

In [ ]:
# # load in the base model
# # freeze all the weights (avoid overfitting)
# cnn_model = VGG16(input_shape = (224, 224, 3),    # Shape of our images
#                     include_top = False,           # Leave out the last fully connected layer
#                     weights = 'imagenet')

# for layer in cnn_model.layers:
#     layer.trainable = False
# cnn_model.layers

In [23]:
# build a list of training data
data_path = f'{DATA_DIR}/raw/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train_resized_augmented'

train_image_data = []
train_label = []

for key, hist in tqdm(hsv_features_train.items()):

    # remove unwanted characters
    img_name_list = key.split("_")[1: -2]
    img_name = "_".join(img_name_list)

    label = df_train[df_train['IMG'] == img_name]['label'].values
    
    if len(label) > 0:
      train_label.append(label[0])
      train_image_data.append(np.array(hist))

x, y, z = train_image_data.shape
train_image_data = np.array(train_image_data).reshape([x, y])
train_label = np.array(train_label)

In [31]:
# build a list of training data
data_path = f'{DATA_DIR}/raw/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test_resized_augmented'

test_image_data = []
test_label = []

for key, hist in tqdm(hsv_features_test.items()):

    # remove unwanted characters
    img_name_list = key.split("_")[1: -2]
    img_name = "_".join(img_name_list)

    label = df_test[df_test['IMG'] == img_name]['label'].values
    
    if len(label) > 0:
      test_label.append(label[0])
      test_image_data.append(np.array(hist))

test_image_data = np.array(test_image_data)
x, y, z = test_image_data.shape
test_image_data = test_image_data.reshape([x, y])
test_label = np.array(test_label)

  0%|          | 0/3119 [00:00<?, ?it/s]

(3119, 256, 1)


In [33]:
import pickle

pickle.dump(train_image_data, open(f"{DATA_DIR}/processed/hsv_hist_train_image_data.pkl", "wb"))
pickle.dump(train_label, open(f"{DATA_DIR}/processed/hsv_hist_train_labels.pkl", "wb"))
pickle.dump(test_image_data, open(f"{DATA_DIR}/processed/hsv_hist_test_image_data.pkl", "wb"))
pickle.dump(test_label, open(f"{DATA_DIR}/processed/hsv_hist_test_labels.pkl", "wb"))